In [1]:
import osmnx as ox
import pandana
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
target = [
    'rogers park', 'west ridge',
    'uptown', 'lincoln square',
    'north center', 'lake view', 'lincoln park',
    'near north side', 'edison park',
    'norwood park', 'jefferson park',
    'forest glen', 'north park', 'albany park',
    'portage park', 'irving park', 'dunning',
    'montclare', 'belmont cragin', 'hermosa',
    'avondale', 'logan square', 'humboldt park',
    'west town', 'austin', 'west garfield park',
    'east garfield park', 'near west side', 
    'north lawndale', 'south lawndale', 'lower west side', 'loop',
    'near south side', 'armour square', 'douglas',
    'oakland', 'fuller park', 'grand boulevard',
    'kenwood', 'washington park', 'hyde park',
    'woodlawn', 'south shore', 'chatham',
    'avalon park', 'south chicago', 'burnside',
    'calumet heights', 'roseland', 'pullman',
    'south deering', 'east side', 'west pullman',
    'riverdale', 'hegewisch', 'garfield ridge',
    'archer heights', 'brighton park', 'mckinley park',
    'bridgeport', 'new city', 'west elsdon',
    'gage park', 'clearing', 'west lawn',
    'chicago lawn', 'west englewood', 'englewood',
    'greater grand crossing', 'ashburn',
    'auburn gresham', 'beverly', 'washington heights',
    'mount greenwood', 'morgan park', 'ohare', 'edgewater'
]

In [3]:
tags = {
    'amenity' : [
        'cafe',
        'bar',
        'restaurant',
        'pub',
        'library',
        'school',
        'university',
        'atm',
        'cinema',
        'nightclub',
        'theatre',
        'police',
        'bench',
        'water_point'
    ],
    'shop' : [
        'bakery',
        'convenience',
        'supermarket',
        'mall',
        'shoes',
        'clothes',
        'department_store',
        'beverages',
        'ice_cream',
        'supermarket',
        'florist',
        'books',
        'anime',
        'laundry',
        'bicycle'
    ],
    'leisure' : [
        'fitness_centre',
        'amusement_arcade',
        'bowling_alley',
        'dog park',
        'fishing',
        'sports_hall',
        'fitness_station',
        'garden',
        'golf_course',
        'park'
    ]
}

In [4]:
# max finder for each col
from collections import deque
def findMax(x):
    queue = []
    max = 0
    for i, num in enumerate(x):
        if num > max:
            max = num
    return max

def findMin(x):
    queue = []
    min = 0
    for i, num in enumerate(x):
        if num > min:
            max = min
    return min

In [5]:
def inverse_distance_score(distances, max_possible_score=10):
    inverse_distances = 1 / distances
    score = inverse_distances.sum() / max_possible_score
    normalized_score = score / inverse_distances.size
    return normalized_score if normalized_score <= 1 else 1

In [6]:
hashmap = {
    'Community Name' : target,
    'Walk_Score' : []
}

In [7]:
for city in target:
    if not city in ["burnside", "riverdale", "mckinley park"]:
        cityname = f"{city}, chicago, illinois"
        crs = 3035
        graph = ox.graph_from_place(cityname, network_type='walk')
        graph = ox.projection.project_graph(graph, to_crs=crs)
        poi = ox.geometries.geometries_from_place(cityname, tags=tags)
        # project the place of interest
        poi = poi.to_crs(epsg=crs)

        max_time = 21
        walk_speed = 4
        for u, v, data in graph.edges(data=True):
            data['speed_kph'] = walk_speed
        graph = ox.add_edge_travel_times(graph)

    
        nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
        edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]

        # Construct the pandana network model
        network = pandana.Network(
            node_x=nodes['x'],
            node_y=nodes['y'], 
            edge_from=edges['u'],
            edge_to=edges['v'],
            edge_weights=edges[['travel_time']]
        )
        centroids = poi.centroid
        """ 
        Calculate and specifiy a max travel distance
        for further implications onto a df we create
        """
        max_distance = max_time * 60 #mins
        network.set_pois(
            category='pois',
            maxdist = max_distance,
            maxitems = 10,
            x_col = centroids.x,
            y_col = centroids.y
        )

        distances = network.nearest_pois(
            distance=max_distance,
            category = 'pois',
            num_pois = 10
        )

        distances.astype(int)
        df = distances
        #(df.iloc[:, 1:] - df.iloc[:, 1:].min()) / (df.iloc[:, 1:].max() - df.iloc[:, 1:].min())
        X = (((df.sum() / df.count()).sum())/10)
        walk_score = X
        hashmap['Walk_Score'].append(walk_score)
    else:
        walk_score = 0
        hashmap['Walk_Score'].append(walk_score)
    

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_2750/3457703710.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2263
Setting CH edge vector of size 7068
Range graph removed 7138 edges of 14136
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_2750/3457703710.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 3090
Setting CH edge vector of size 9804
Range graph removed 9946 edges of 19608
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_2750/3457703710.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


...

Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2855
Setting CH edge vector of size 9414
Range graph removed 9522 edges of 18828
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_2750/3457703710.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 1628
Setting CH edge vector of size 4132
Range graph removed 4400 edges of 8264
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_2750/3457703710.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2593
Setting CH edge vector of size 8686
Range graph removed 8792 edges of 17372
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [8]:
import pandas as pd
len(hashmap['Community Name'])
len(hashmap["Walk_Score"][1:])

76

In [18]:
df_poi = pd.DataFrame(hashmap)

In [9]:
df.min().max()

67.4000015258789

In [10]:
(((df.sum() / df.count()).sum())/10)

236.47185232139122

In [11]:
df[10].mean()

330.21695472078807

In [12]:
df_max = df.max()
df_min = df.min()

In [13]:
x = findMax(df_max)
y = findMin(df_min)
X_std = (df - df.min(axis=0)) / (df.max(axis=0)) - df.min(axis=0)
X_scaled = X_std * (x - y) + y
print(X_scaled)

                     1           2           3           4           5   \
osmid                                                                     
104509535     86.254069  241.653981  291.081671  323.260841  412.422774   
4065425671    58.217030  234.248186  270.031530  303.208692  393.333616   
4065557345   105.362115  251.474680  305.427931  336.926922  425.432550   
5496651199   154.114421  180.475748  342.031018  352.253394  368.286708   
1257584212   107.505075  265.481268  307.036854  338.459585  426.891608   
...                 ...         ...         ...         ...         ...   
11601309417  208.581278  215.572720  376.354797  400.148618  380.931738   
11607092434  388.768342  375.602079  455.058106  433.994576  534.860759   
11607092458  282.691918  425.671608  392.578109  433.739159  479.538720   
11607108111  127.506020  233.443210  277.672598  275.876492  382.511165   
11607108112  125.898799  234.570172  257.292864  274.342562  363.180080   

                     6  

In [ ]:
df_poi.isnull().sum()
df_poi.to_csv('ura_data_processed.csv')
print(f'Saved df_poi')